## 1. 데이터 전처리

In [5]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"
%pip install openpyxl


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

In [7]:
train_df = pd.read_excel('통합 데이터 셋_3800.xlsx')
test_df = pd.read_excel('테스트 셋_800 완료.xlsx')

In [9]:
train_df.head()

,관리번호,설비번호(구),설비명,수리일자,고장내용,고장부품,불량유형,조치내용,수리처,수리시간(분),수리금액
0,S000000001,9,성형연삭기,2021-01-21 00:00:00,출하장 입구 에어 50A 유니온 에어리크 재조임,출하장 입구 에어 50A 유니온,에어리크,재조임,오정수,40,NaN
1,S000000001,9,성형연삭기,2021-02-23 00:00:00,"SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공",SHIELD ROOM 화풍기 철거부위,"마감, 판넬벽체문 타공",마감 및 철거부위 수정,오정수윤동태젱알,120,NaN
2,S000000001,9,성형연삭기,2021-03-03 00:00:00,검사반 소형 랩 포장기 히팅안됨 점검 수리불가,소형 랩 포장기,히팅 불능,점검 후 수리 불가,오정수윤동태,40,NaN
3,S000000001,9,성형연삭기,2021-03-03 00:00:00,기준과절차준수동 수입검사쪽 자동문 안됨임시개방,자동문,작동불량,임시 개방,오정수윤동태,30,NaN
4,S000000001,9,성형연삭기,2021-03-03 00:00:00,보관 여자 화장실 전등자동감지기 누전소손 신규교체,보관 여자 화장실 전등자동감지기,누전소손,신규교체,송명호,181,NaN


In [10]:
test_df.head()

,관리번호,설비번호(구),설비명,수리일자,고장내용,고장부품,불량유형,조치내용,수리처,수리시간(분),수리금액
0,Z000000001,Z000000001,기타,2023-01-03,3공장렉 철거후 2공장으로 이동,3공장 렉,철거,2공장으로 이동,오정수이승희,480,NaN
1,Z000000001,Z000000001,기타,2023-01-03,CN7라인 천정 전등 교체,CN7라인 천정 전등,고장,전등 교체,송명호윤동태,180,NaN
2,Z000000001,Z000000001,기타,2023-01-03,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,E4L 인너카바 리벳툴,진공이상,리벳 잔영물 제거 청소,최혁진윤동태,30,NaN
3,Z000000001,Z000000001,기타,2023-01-03,전착도장 대차 제작중,전착도장 대차,NaN,제작중,김정표오정수,140,NaN
4,M000000038,ASM-029,조립기(DH 스위치 조립기),2023-01-04,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,#40 LH 트램스파 전후진 솔밸브,동작이상,단선 수리,송명호윤동태,150,NaN


In [16]:
from datasets import Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [17]:
def format_data(sample):
    # 고장부품, 불량유형, 조치내용을 딕셔너리 형태로 구성
    response_dict = {
        "고장부품": sample["고장부품"],
        "불량유형": sample["불량유형"], 
        "조치내용": sample["조치내용"]
    }
    
    return {
        "messages": [
            {
                "role": "system",
                "content": '''주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.''',
            },
            {
                "role": "user",
                "content": f"고장 내용: {sample['고장내용']}",
            },
            {
                "role": "assistant",
                "content": str(response_dict)  # 나머지 3개 필드를 딕셔너리로 변환하여 assistant 응답으로
            },
        ],
    }

# train_df와 test_df를 OpenAI format으로 변환
train_dataset = []
for _, row in train_df.iterrows():
    train_dataset.append(format_data(row))

test_dataset = []
for _, row in test_df.iterrows():
    test_dataset.append(format_data(row))

# 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")


전체 데이터 분할 결과: Train 3801개, Test 800개


In [18]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.\n이때 파이썬의 Dictionary 형태로 반환하시오.'},
 {'role': 'user',
  'content': '고장 내용: 130공정 소재배출 ng알람  p245릴레이굧[,30.50공정 조정 수리'},
 {'role': 'assistant',
  'content': "{'고장부품': '130공정 소재배출 p245릴레이', '불량유형': 'NG 알람', '조치내용': '[,30.50공정 조정과 관련된 수리'}"}]

In [19]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [20]:
train_dataset[0]

{'messages': [{'content': '주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.\n이때 파이썬의 Dictionary 형태로 반환하시오.',
   'role': 'system'},
  {'content': '고장 내용: 출하장 입구 에어 50A 유니온 에어리크 재조임', 'role': 'user'},
  {'content': "{'고장부품': '출하장 입구 에어 50A 유니온', '불량유형': '에어리크', '조치내용': '재조임'}",
   'role': 'assistant'}]}

## 2. 모델 로드 및 템플릿 적용

In [22]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2.5-0.5B-Instruct" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [23]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 출하장 입구 에어 50A 유니온 에어리크 재조임<|im_end|>
<|im_start|>assistant
{'고장부품': '출하장 입구 에어 50A 유니온', '불량유형': '에어리크', '조치내용': '재조임'}<|im_end|>



## 3. LoRA와 SFTConfig 설정

In [24]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

`lora_alpha`: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.

`lora_dropout`: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. `0.1`로 설정하면 학습 중 10%의 뉴런이 비활성화됩니다.

`r`: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.

`bias`: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. `"none"`으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. `"all"` 또는 `"lora_only"`와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.

`target_modules`: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 `"q_proj"`와 `"v_proj"`를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.

`task_type`: LoRA가 적용되는 작업 유형을 지정합니다. `"CAUSAL_LM"`은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 `"SEQ2SEQ_LM"`(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [25]:
args = SFTConfig(
    output_dir="qwen2.5-0.5b",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

`output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

`num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니다.

`per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있습니다.

`gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 2로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 유용합니다.

`gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질 수 있습니다.

`optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적화기를 사용합니다.

`logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마다 로그를 기록합니다.

`save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스텝마다 모델이 저장됩니다.

`save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50 스텝마다 모델을 저장합니다.

`bf16`: bfloat16 정밀도를 사용하도록 설정합니다. bfloat16은 FP32와 유사한 범위를 제공하면서 메모리와 계산 효율성을 높입니다.

`learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 사용하여 안정적인 학습을 유도합니다.

`max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 조정하여 폭발적 그래디언트를 방지합니다.

`warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다. 학습의 안정성을 높이기 위해 사용됩니다.

`lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"constant"`는 학습률을 일정하게 유지합니다.

`push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.

`remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다. True로 설정하면 메모리를 절약할 수 있습니다.

`dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset: True`로 설정하여 데이터셋 준비 단계를 건너뜹니다.

`report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn

In [26]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

`collate_fn(batch)` 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 레이블 데이터를 초기화한 다음, 특정 토큰 패턴(<|im_start|>assistant 이후부터 <|im_end|>까지)을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

최종적으로, 배치 내 모든 샘플의 길이를 동일하게 맞추기 위해 패딩 작업을 수행합니다. 이 과정에서 입력 데이터에는 패딩 토큰 ID를 추가하고, 어텐션 마스크에는 0을 추가하며, 레이블에는 -100을 추가합니다. 모든 데이터는 PyTorch 텐서로 변환되어 반환됩니다.

In [27]:
# 최대 길이
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 127])
어텐션 마스크 형태: torch.Size([1, 127])
레이블 형태: torch.Size([1, 127])


In [28]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 54330, 31079, 85251, 126429, 40853, 95218, 26699, 10764, 44104, 53189, 138020, 126429, 40853, 63089, 125678, 11, 126488, 131837, 125522, 128909, 11, 65510, 59698, 131193, 17877, 57835, 69923, 127268, 57268, 624, 12802, 131866, 54969, 12802, 144773, 20401, 10466, 141966, 17380, 94879, 127268, 57268, 13, 151645, 198, 151644, 872, 198, 34395, 40853, 130213, 25, 36330, 250, 16186, 40853, 38150, 88259, 90486, 31079, 220, 20, 15, 32, 126310, 83036, 130000, 90486, 31079, 28002, 81133, 129242, 92817, 93701, 151645, 198, 151644, 77091, 198, 13608, 34395, 40853, 63089, 125678, 1210, 364, 69923, 16186, 40853, 38150, 88259, 90486, 31079, 220, 20, 15, 32, 126310, 83036, 130000, 516, 364, 134836, 131837, 125522, 128909, 1210, 364, 19391, 31079, 28002, 81133, 516, 364, 140050, 95218, 26699, 1210, 364, 57132, 92817, 93701, 8275, 151645]


In [29]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 198, 13608, 34395, 40853, 63089, 125678, 1210, 364, 69923, 16186, 40853, 38150, 88259, 90486, 31079, 220, 20, 15, 32, 126310, 83036, 130000, 516, 364, 134836, 131837, 125522, 128909, 1210, 364, 19391, 31079, 28002, 81133, 516, 364, 140050, 95218, 26699, 1210, 364, 57132, 92817, 93701, 8275, 151645]


LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 이를 예시 문장과 정수 인코딩을 통해 상세히 설명하겠습니다.

---

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다.  
사용자가 **"안녕하세요, 오늘 날씨는 어떤가요?"**라고 물었고,  
모델은 **"안녕하세요! 오늘 날씨는 맑고 화창합니다."**라고 응답해야 한다고 합시다.

이 데이터를 학습하기 위해 먼저 전체 대화 데이터를 정수로 인코딩합니다.  
토크나이저는 대화의 구조를 구분하기 위해 `<|im_start|>`와 `<|im_end|>`을 사용하여 정수 시퀀스를 생성한다고 가정해봅시다.  
(실제로는 LLM 템플릿이 이보다는 복잡함을 기억하고 혼동하지 맙시다.)
정수 시퀀스는 다음과 같이 구성될 수 있습니다.  

---
<|im_start|>user 안녕하세요, 오늘 날씨는 어떤가요?<|im_end|>  
<|im_start|>assistant 안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>

---

이를 정수로 변환하면 다음과 같습니다.  
`input_ids = [1001, 2001, 3001, 4001, 5001, 6001, 7001, 1002, 1001, 8001, 9001, 1003, 2002]`  
모델이 예측해야 하는 부분은 `assistant`의 응답인 "안녕하세요! 오늘 날씨는 맑고 화창합니다."입니다. 따라서, `labels`는 다음과 같이 설정됩니다.

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, 8001, 9001, 1003, 2002]`  

이처럼 `labels`는 모델의 출력이 필요한 영역만을 포함하고, 나머지 부분은 `-100`으로 채워져 모델이 실제로 예측하고 오차를 계산해야 하는 대상(학습 대상)에서 제외됩니다. 이를 통해 모델은 불필요한 영역을 학습하지 않고, 필요한 응답 영역에만 집중할 수 있습니다.


## 5. 학습

In [30]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [31]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.889000
20,0.509100
30,0.474700
40,0.435200
50,0.517400
60,0.445700
70,0.404300
80,0.428800
90,0.439600
100,0.367300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

## 6. 테스트 데이터 준비

실제 모델에 입력을 넣을 때에는 입력의 뒤에 '<|im_start|>assistant'가 부착되어서 넣는 것이 좋습니다. 그래야만 모델이 바로 답변을 생성합니다.

In [32]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [33]:
print(prompt_lst[42])

<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 생산수량 간헐적 수량 누락 카운터 단자 재 조임<|im_end|>
<|im_start|>assistant


In [34]:
print(label_lst[42])


{'고장부품': '카운터 단자', '불량유형': '생산 수량 간헐적 누락', '조치내용': '카운터 단자 재 조임'}<|im_end|>



## 7. 파인 튜닝 모델 테스트

`AutoPeftModelForCausalLM()`의 입력으로 LoRA Adapter가 저장된 체크포인트의 주소를 넣으면 LoRA Adapter가 기존의 LLM과 부착되어 로드됩니다. 이 과정은 LoRA Adapter의 가중치를 사전 학습된 언어 모델(LLM)에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.

`peft_model_id` 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. `"qwen2-7b-rag-ko/checkpoint-285"`는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서 해당 가중치를 불러옵니다.

`fine_tuned_model`은 `AutoPeftModelForCausalLM.from_pretrained` 메서드를 통해 체크포인트를 로드하여 생성됩니다. 이 메서드는 LLM과 LoRA Adapter를 결합하고, 최적화된 설정으로 모델을 메모리에 로드합니다. `device_map="auto"` 옵션은 모델을 자동으로 GPU에 배치합니다.

`pipeline`은 Hugging Face의 고수준 유틸리티로, NLP 작업(예: 텍스트 생성, 번역, 요약 등)을 간단히 수행할 수 있게 해줍니다. 이 코드에서 사용된 `pipeline("text-generation")`은 텍스트 생성 작업을 수행하기 위한 파이프라인 객체를 생성합니다. 파이프라인은 내부적으로 모델과 토크나이저를 관리하여, 입력 텍스트를 토큰화하고, 모델을 통해 생성된 결과를 다시 디코딩하여 사람이 읽을 수 있는 텍스트로 변환합니다.

이 코드는 미세 조정된 LLM을 로드한 뒤, 이를 이용해 텍스트 생성 작업을 간단히 수행할 수 있도록 준비하는 데 목적이 있습니다. `pipeline`을 통해 텍스트 생성 작업을 실행하면, 입력 텍스트에 기반하여 모델이 다음 토큰을 예측하고 이를 반복적으로 생성합니다. 이 과정은 사용자에게 자연스러운 텍스트를 출력하는 데 사용됩니다.

In [35]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [57]:
peft_model_id = "qwen2.5-0.5b/checkpoint-950"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [58]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [59]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [56]:
for prompt, label in zip(prompt_lst[20:25], label_lst[20:25]):
    # print(f"    prompt:\n{prompt}")
    print(f"    input:\n{prompt}")
    print("-"*50)
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("="*50)

    input:
<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 전착행거 대차 제작<|im_end|>
<|im_start|>assistant
--------------------------------------------------
    response:
{'고장부품': '전착행거 대차', '불량유형': '제작 필요', '조치내용': '대차 제작'}
    label:

{'고장부품': '전착행거 대차', '불량유형': '제작불량', '조치내용': '전착행거 대차 제작'}<|im_end|>

    input:
<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 파렛트 바퀴 수리,잠금고리 수리<|im_end|>
<|im_start|>assistant
--------------------------------------------------
    response:
{'고장부품': '파렛트 바퀴, 잠금고리', '불량유형': '고장', '조치내용': '바퀴 수리 및 잠금고리 수리'}
    label:

{'고장부품': '파렛트 바퀴, 잠금고리', '불량유형': '고장', '조치내용': '바퀴, 잠금고리 수리'}<|im_end|>

    input:
<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 코킹감지센서 단선 수리<|im_end|>
<|im_start|>assistant
-----------------------------------------

## 8. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.

In [61]:
base_model_id = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [62]:
for prompt, label in zip(prompt_lst[20:25], label_lst[20:25]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
```python
{"고장부품": "전착행거 대차 제작", "불량유형": "기타", "조치사항": "상황에 따라 개선 또는 재구성"}
```
    label:

{'고장부품': '전착행거 대차', '불량유형': '제작불량', '조치내용': '전착행거 대차 제작'}<|im_end|>

--------------------------------------------------
    response:
```python
{
    "고장부품": ["파lets", "잠금"],
    "불량유형": "기타",
    "조치사항": "수리"
}
```
    label:

{'고장부품': '파렛트 바퀴, 잠금고리', '불량유형': '고장', '조치내용': '바퀴, 잠금고리 수리'}<|im_end|>

--------------------------------------------------
    response:
```python
{
    "고장부품": ["코킹감지센서"],
    "불량유형": "단선",
    "조치사항": "수정 필요"
}
```
    label:

{'고장부품': '코킹감지센서', '불량유형': '단선', '조치내용': '센서 수리'}<|im_end|>

--------------------------------------------------
    response:
```python
{
    "고장부품": ["클램프"],
    "불량유형": "CLAMP",
    "조치사항": "클램프를 OPEN 상태로 하강하고, 실린더 높이를 조정합니다."
}
```
    label:

{'고장부품': '클램프', '불량유형': '안착 후 OPEN 불량', '조치내용': '하강실린더 높이 조정'}<|im_end|>

--------------------------------------------------
    response:
```python
{
    "고장부품": ["단선 재납"],
    "불량유형": ["단선